In [3]:
import pandas as pd

In [4]:
crime_data = pd.read_csv('Crime_Data_from_2020_to_present.csv')

In [4]:
crime_data.head()

,DR_NO,Date Rptd,DATE OCC,TIME OCC,AREA,AREA NAME,Rpt Dist No,Part 1-2,Crm Cd,Crm Cd Desc,...,Status,Status Desc,Crm Cd 1,Crm Cd 2,Crm Cd 3,Crm Cd 4,LOCATION,Cross Street,LAT,LON
0,10304468,01/08/2020 12:00:00 AM,01/08/2020 12:00:00 AM,2230,3,Southwest,377,2,624,BATTERY - SIMPLE ASSAULT,...,AO,Adult Other,624.0,NaN,NaN,NaN,1100 W 39TH PL,NaN,34.0141,-118.2978
1,190101086,01/02/2020 12:00:00 AM,01/01/2020 12:00:00 AM,330,1,Central,163,2,624,BATTERY - SIMPLE ASSAULT,...,IC,Invest Cont,624.0,NaN,NaN,NaN,700 S HILL ST,NaN,34.0459,-118.2545
2,200110444,04/14/2020 12:00:00 AM,02/13/2020 12:00:00 AM,1200,1,Central,155,2,845,SEX OFFENDER REGISTRANT OUT OF COMPLIANCE,...,AA,Adult Arrest,845.0,NaN,NaN,NaN,200 E 6TH ST,NaN,34.0448,-118.2474
3,191501505,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,1730,15,N Hollywood,1543,2,745,VANDALISM - MISDEAMEANOR ($399 OR UNDER),...,IC,Invest Cont,745.0,998.0,NaN,NaN,5400 CORTEEN PL,NaN,34.1685,-118.4019
4,191921269,01/01/2020 12:00:00 AM,01/01/2020 12:00:00 AM,415,19,Mission,1998,2,740,"VANDALISM - FELONY ($400 & OVER, ALL CHURCH VA...",...,IC,Invest Cont,740.0,NaN,NaN,NaN,14400 TITUS ST,NaN,34.2198,-118.4468


In [5]:
crime_data.count()

DR_NO             761582
Date Rptd         761582
DATE OCC          761582
TIME OCC          761582
AREA              761582
AREA NAME         761582
Rpt Dist No       761582
Part 1-2          761582
Crm Cd            761582
Crm Cd Desc       761582
Mocodes           657013
Vict Age          761582
Vict Sex          662009
Vict Descent      662003
Premis Cd         761573
Premis Desc       761138
Weapon Used Cd    264752
Weapon Desc       264752
Status            761582
Status Desc       761582
Crm Cd 1          761572
Crm Cd 2           56342
Crm Cd 3            1875
Crm Cd 4              57
LOCATION          761582
Cross Street      122391
LAT               761582
LON               761582
dtype: int64

In [15]:

str(crime_data['TIME OCC'][0])
str(crime_data['TIME OCC'][0])[-3]
j = f"asdf{1}"
j

'asdf1'

In [11]:
times = []
for time in crime_data['TIME OCC']:
    time_str = str(time)
    if len(time_str) == 1:
        converted_time = f"00:0{time_str[-1]}:00"
    elif len(time_str) == 2:
        converted_time = f"00:{time_str[-2:]}:00"
    elif len(time_str) == 3:
        converted_time = f"0{time_str[-3]}:{time_str[-2:]}:00"
    else:
        converted_time = f"{time_str[-4:-2]}:{time_str[-2:]}:00"
    times.append(converted_time)


In [12]:
#pd.to_datetime(times[0:6])
#pd.to_datetime(crime_data['DATE OCC']. + times)
pd.to_datetime(pd.to_datetime(crime_data['DATE OCC']).astype(str) + ' ' + times)

0        2020-01-08 22:30:00
1        2020-01-01 03:30:00
2        2020-02-13 12:00:00
3        2020-01-01 17:30:00
4        2020-01-01 04:15:00
                 ...        
761577   2023-01-26 18:00:00
761578   2023-03-22 10:00:00
761579   2023-04-12 16:30:00
761580   2023-07-01 00:01:00
761581   2023-03-05 09:00:00
Name: DATE OCC, Length: 761582, dtype: datetime64[ns]